In [1]:
import numpy as np
import pandas as pd

from math import ceil
from numba import njit, prange
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_array
from pyts.approximation import PiecewiseAggregateApproximation
from keras.utils import np_utils

import os
import cv2
import tensorflow as tf

In [2]:
# npz file import

Welding_data = np.load('E:/Result/ver.3.21/MTF/MTF.npz')

X_data = Welding_data['X_data']
y_data = Welding_data['y_data']
i_data = Welding_data['i_data']

Welding_data.close()

In [3]:
seed = 7
np.random.seed()

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, i_train, i_test = train_test_split(X_data, y_data, i_data, test_size= 0.2, shuffle= True, random_state= seed)

In [4]:
inputs = np.concatenate((X_train, X_test))
targets = np.concatenate((y_train, y_test))
index = np.concatenate((i_train, i_test))

In [5]:
targets = np_utils.to_categorical(targets)

In [6]:
# k-fold cross validation

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

kfold = KFold(n_splits= 10, shuffle= True, random_state= seed)

test= []
train= []
test_= []
train_ =[]

for train, test in kfold.split(inputs, targets):
    print(len(test))
    print(len(train))
    
    print('-----------')

    for i in zip(test):
        test_.append(i)
        
    for i in zip(train):
        train_.append(i)
        

300
2700
-----------
300
2700
-----------
300
2700
-----------
300
2700
-----------
300
2700
-----------
300
2700
-----------
300
2700
-----------
300
2700
-----------
300
2700
-----------
300
2700
-----------


In [7]:
train = train_[0:2700]
train = np.reshape(train, 2700)
test = test_[0:300]
test = np.reshape(test, 300)

In [8]:
print(inputs[train].shape)
print(targets[train].shape)

(2700, 300, 300, 2)
(2700, 3)


In [9]:
print(inputs[test].shape)

(300, 300, 300, 2)


In [10]:
from keras.models import load_model

with tf.device('/gpu:0'):
    model = load_model('E:/Result/ver.3.21/MTF/weight/MTF_1.h5')
    y_predict = model.predict(inputs[test])
    print(y_predict.shape)
    y_predict_Reshape = np.reshape(y_predict, 300)
    
    list_y_predict_ = []
    
    for j in range(len(y_predict_Reshape)):
        y_predict_Round = round(y_predict_Reshape[j], 0)
        y_predict_int = int(y_predict_Round)
        list_y_predict_.append(y_predict_int)
        
    y_predict = np.array(list_y_predict_)
    print(y_predict)

(300, 3)


ValueError: cannot reshape array of size 900 into shape (300,)

In [11]:
y_predict = np.reshape(y_predict, (300,3))
y_predict = np.argmax(y_predict, axis= 1)

In [12]:
targets = np.argmax(targets[test], axis= 1)

In [13]:
print(targets.shape)

(300,)


In [14]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

Acc = accuracy_score(targets, y_predict)
print("ACC : ", Acc)

Precision = precision_score(targets, y_predict, average= "macro")
print("Precision :", Precision)

Recall = recall_score(targets, y_predict, average= "macro")
print("Recall : ", Recall)

f1 = f1_score(targets, y_predict, average= "macro")
print("f1 :", f1)

ACC :  0.9866666666666667
Precision : 0.9863433667781494
Recall :  0.9863433667781494
f1 : 0.9863433667781494


In [15]:
from sklearn.metrics import precision_recall_fscore_support as score

Precision, Recall, f1score, support = score(targets, y_predict)

print('Precision : {}'.format(Precision))
print('Recall : {}'.format(Recall))
print('f1score : {}'.format(f1score))
print('Accuracy : {}'.format(support))

Precision : [1.         0.97826087 0.98076923]
Recall : [1.         0.97826087 0.98076923]
f1score : [1.         0.97826087 0.98076923]
Accuracy : [104  92 104]


In [16]:
X_incorrect = []
X_correct = []
y_incorrect = []
i_incorrect = []
y_correct = []
i_correct = []
y_pre_co = []
y_pre_inco = []

for i in range(0,len(targets)):
    if y_predict[i] != targets[i]:
        X_incorrect.append(inputs[test][i])
        y_incorrect.append(targets[i])
        i_incorrect.append(index[test][i])
        y_pre_inco.append(y_predict[i])
    else:
        X_correct.append(inputs[test][i])
        y_correct.append(targets[i])
        i_correct.append(index[test][i])
        y_pre_co.append(y_predict[i])

MemoryError: Unable to allocate 206. MiB for an array with shape (300, 300, 300, 2) and data type float32